In [1]:
## Correcting recent leads

In [9]:
# Parameters
api_url = '/services/data/v43.0/sobjects'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"npass2021HDfJLhBGDx11xWCKlEbHQKhF",    
}
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [10]:
import pandas as pd
import datetime

#stagnant reports
import os
cwd = os.getcwd()

In [11]:
pd.set_option('display.max_columns', 500)

In [12]:
#input new filename here
filename = "export_13650230[1].csv"
f_type = filename.split('.')[-1]

In [13]:
#pandas import
if f_type == 'csv':
    idf = pd.read_csv(cwd+'/Data Files/'+filename)#,encoding="cp1252")#,encoding='utf-16',sep='\t')
                      #,encoding="cp1252",error_bad_lines=False)#,usecols=[12])#col_fields)
elif f_type == 'xlsx' or f_type == 'xls':
    xl = pd.ExcelFile(cwd+'/Data Files/'+filename)
    print(xl.sheet_names)# see all sheet names
    sheet_names = xl.parse(xl.sheet_names) #this already performs an import
    
    idf = pd.read_excel(cwd+'/Data Files/'+filename,sheet_name=0,header=0,skiprows=1)

print(idf.head(2))

       Subscriber Key                   Email Address
0  00Q5w00001tLZabEAG      309602099xinyang@gmail.com
1  00Q5w00001tLZacEAG  sp1709@scarletmail.rutgers.edu


In [14]:
idf = idf.rename(columns={'Subscriber Key':'Id','Email Address':'Email'})
display(idf.head(2))

,Id,Email
0,00Q5w00001tLZabEAG,309602099xinyang@gmail.com
1,00Q5w00001tLZacEAG,sp1709@scarletmail.rutgers.edu


In [15]:
idf['Venue__c'] = 'Citrine Webinar 10/21/20 lead_registrant'

In [20]:
print(idf.shape)

(214, 3)


In [16]:
# obtain list of all salesforce campaigns
from DB2SalesforceAPI import DB2SalesforceAPI
# obtain all campaign names
db_c = DB2SalesforceAPI(sf_login_params)
mod_lead = db_c.query_data('SELECT Id, SF_lead_indexer__c, FirstName, LastName, LastModifiedById, Venue__c FROM Lead')

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000Wx0WNAAZ
{"id":"7505w00000Wx0WNAAZ","operation":"query","object":"Lead","createdById":"0055w00000DM5bOAAT","createdDate":"2021-06-15T17:16:49.000+0000","systemModstamp":"2021-06-15T17:16:49.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000Wx0WNAAZ","operation":"query","object":"Lead","createdById":"0055w00000DM5bOAAT","createdDate":"2021-06-15T17:16:49.000+0000","systemModstamp":"2021-06-15T17:16:58.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000Wx0WNAAZ","operation":"query","object":"Lead","createdById":"0055w00000DM5bOAAT

In [17]:
display(mod_lead.head(2))

,FirstName,Id,LastModifiedById,LastName,SF_lead_indexer__c,Venue__c
0,Alex,00Q5w00001otOM1EAM,0055w00000ArpYvAAJ,Wolf (Sample),1,NaN
1,John,00Q5w00001otOM2EAM,0055w00000ArpYvAAJ,Steele (Sample),2,NaN


In [19]:
matches = []
#r_ln = r_df['LastName'].to_list()
for ind,val in enumerate(mod_lead['Id'].to_list()):
    if val in idf['Id'].to_list():
        matches.append(ind)
print(len(matches))

214


In [21]:
mod_lead2 = mod_lead.iloc[matches,:].reset_index()
display(mod_lead2.head(2))
display(mod_lead2.shape)

,index,FirstName,Id,LastModifiedById,LastName,SF_lead_indexer__c,Venue__c
0,6546,Srini,00Q5w00001tLZaJEAW,0055w00000CX7FfAAL,Raghavan,6547,Citrine Material Science workshop - R
1,6547,Roberta,00Q5w00001tLZaKEAW,0055w00000DM5bOAAT,Cappabianca,6548,Citrine Material Science workshop - R


(214, 7)

In [22]:
mod_lead2['Venue__c']

0                  Citrine Material Science workshop - R
1                  Citrine Material Science workshop - R
2                  Citrine Material Science workshop - R
3                  Citrine Material Science workshop - R
4      Citrine Material Science workshop - R;Citrine ...
                             ...                        
209                Citrine Material Science workshop - R
210                Citrine Material Science workshop - R
211                Citrine Material Science workshop - R
212                Citrine Material Science workshop - R
213                Citrine Material Science workshop - R
Name: Venue__c, Length: 214, dtype: object

In [28]:
tvc = mod_lead2['Venue__c'].to_list()
from copy import deepcopy
for i,j in enumerate(tvc):
    j = j+';Citrine Webinar 10/21/20 lead_registrant'
    tj = j.split(';')
    tj2 = set(tj)
    j = ';'.join(tj2)
    tvc[i] = deepcopy(j)
mod_lead2['Venue__c'] = tvc

In [29]:
display(mod_lead2.head(2))

,index,FirstName,Id,LastModifiedById,LastName,SF_lead_indexer__c,Venue__c
0,6546,Srini,00Q5w00001tLZaJEAW,0055w00000CX7FfAAL,Raghavan,6547,Citrine Material Science workshop - R;Citrine ...
1,6547,Roberta,00Q5w00001tLZaKEAW,0055w00000DM5bOAAT,Cappabianca,6548,Citrine Material Science workshop - R;Citrine ...


In [34]:
mod_lead2 = mod_lead2.drop(columns=['LastModifiedById'])#,'index','Id'])
display(mod_lead2.head(2))

,index,FirstName,Id,LastName,SF_lead_indexer__c,Venue__c
0,6546,Srini,00Q5w00001tLZaJEAW,Raghavan,6547,Citrine Material Science workshop - R;Citrine ...
1,6547,Roberta,00Q5w00001tLZaKEAW,Cappabianca,6548,Citrine Material Science workshop - R;Citrine ...


In [39]:
mod_lead2 = mod_lead2.drop(columns=['index','FirstName','Id','LastName'])

In [60]:
mod_lead2.head(2)
mod_lead2.shape

(214, 2)

In [49]:
db_c.external_id = 'SF_lead_indexer__c'
db_c.object_id = 'Lead'
db_c.send_data(mod_lead2)

[Success] Bulk job creation successful. Job ID = 7505w00000Wx0ZWAAZ
[Success] CSV upload successful. Job ID = 7505w00000Wx0ZWAAZ
[Success] Closing job successful. Job ID = 7505w00000Wx0ZWAAZ


In [63]:
db_c.check_bulk_status()

{'id': '7505w00000Wx0ZWAAZ',
 'operation': 'upsert',
 'object': 'Lead',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-06-15T17:24:05.000+0000',
 'systemModstamp': '2021-06-15T17:25:45.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'SF_lead_indexer__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 214,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 734,
 'apiActiveProcessingTime': 576,
 'apexProcessingTime': 106}

In [64]:
from pprint import pprint

pprint(db_c.check_bulk_failed_results())

'"sf__Id","sf__Error",SF_lead_indexer__c,Venue__c\n'
